In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("../..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


### Access Operational DB (Post Analyics) -> CosmosDB

In [2]:
from src.cosmosdb.manager import CosmosDBMongoCoreManager

cosmos_manager = CosmosDBMongoCoreManager(
    connection_string=os.getenv("AZURE_COSMOS_CONNECTION_STRING"),
    database_name=os.getenv("AZURE_COSMOS_DB_DATABASE_NAME"),
    collection_name=os.getenv("AZURE_COSMOS_DB_COLLECTION_NAME"),
)

c:\Users\pablosal\Desktop\gbb-ai-audio-agent\src\cosmosdb\manager.py:31: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client = pymongo.MongoClient(connection_string)


In [8]:
query = {"session_id": "a23455d6"}
document = cosmos_manager.read_document(query)

if document:
    print("Found session:", document)
else:
    print("No session found.")

ERROR:src.cosmosdb.manager:Failed to read document: Request blocked by network firewall, full error: {'ok': 0.0, 'errmsg': 'Request blocked by network firewall', 'code': 18, 'codeName': 'AuthenticationFailed'}, Timeout: 30s, Topology Description: <TopologyDescription id: 68361aaeb71b114712449cdb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('cosmosdb-ai-factory-westus2.mongo.cosmos.azure.com', 10255) server_type: Unknown, rtt: None, error=OperationFailure("Request blocked by network firewall, full error: {'ok': 0.0, 'errmsg': 'Request blocked by network firewall', 'code': 18, 'codeName': 'AuthenticationFailed'}")>]>


No session found.


## Access Cache (redis) 

In [2]:
from src.stateful.state_managment import (
    MemoManager,
)
from src.redis.manager import AzureRedisManager

redis_manager = AzureRedisManager()

2025-07-22 10:35:12,787 - micro - MainProcess - INFO     Azure Redis connection initialized with access key. (manager.py:_create_client:101)
INFO:micro:Azure Redis connection initialized with access key.


In [7]:
cm = MemoManager.from_redis("bda61aff", redis_manager)

In [8]:
cm.context

{'tool_outputs': {'authenticate_caller': {'authenticated': True,
   'message': 'Authenticated Alice Brown.',
   'policy_id': 'POL-A10001',
   'caller_name': 'Alice Brown',
   'attempt': 2,
   'intent': 'claims',
   'claim_intent': 'new_claim'},
  'handoff_general_agent': {'ok': True,
   'message': 'Caller transferred to General Insurance Questions agent.',
   'data': {'handoff': 'ai_agent',
    'target_agent': 'General Insurance Questions',
    'topic': 'policy'}}},
 'authenticated': True,
 'caller_name': 'Alice Brown',
 'policy_id': 'POL-A10001',
 'call_reason': None,
 'claim_intent': 'new_claim',
 'active_agent': 'Claims'}

In [9]:
cm.histories

{'AuthAgent': [{'role': 'system',
   'content': '\n\nRTAgent – Authentication + Smart Routing (Insurance Voice Bot)\nTriggered BEFORE any service actions are permitted\n\n\n"""\nTask\n"""\nYou are XYMZ Insurance’s voice agent, running as one low-latency LLM turn in the STT ➜ LLM ➜ TTS pipeline.\nSound and think like a caring human who can connect the dots—even when the caller is upset, uncertain, or switching languages.\n\nDuring **Step 1 of every call** you MUST accomplish two things:\n\n| Step                | What you do                                                                 | Mandatory tool call         |\n|---------------------|-----------------------------------------------------------------------------|----------------------------|\n| 1 · Discover intent | Greet the caller. Save their reason verbatim as call_reason. Classify that   | none                       |\n|                     | reason: intent = "claims" or "general". If intent == "claims", also set     |       

In [5]:
cm.histories["IntentClassifierAgent"]

[{'role': 'system',
  'content': '\n\n"""ROLE"""\nYou are a healthcare voice intent classifier.\nYour job: **analyze the user\'s most recent utterance and output a single JSON object** with:\n  - category (from list below, see expanded guidance)\n  - intent (most specific supported intent, e.g., "refill_prescription")\n  - slots (dictionary with any clearly specified details, e.g., drug_name, pharmacy, date)\n\n"""CATEGORIES & DECISION GUIDANCE"""\n- **Scheduling**: \n    - Appointments for check-ups, follow-ups, annual exams, immunizations, primary care.\n    - Includes requests like: “I need to book a checkup”, “Cancel my appointment”, “Change my follow-up date”, “What appointments do I have coming up?”\n    - If the request is for a **specialist** or contains a specialty (e.g., "cardiologist"), use **Referrals**.\n    - If scheduling combines routine and specialist, prefer **Referrals**.\n- **Medication**:\n    - Prescription refills, new prescriptions, getting a list of medications

In [17]:
cm.hist[:100]

[{'role': 'system',
  'content': '\n\n"""ROLE & CONTEXT"""\nYou are RTMedAgent, a HIPAA‑compliant, real‑time healthcare voice assistant\nserving patients of XYZ Health Clinic after multi‑factor authentication.\n\n"""CALL METADATA"""\n• patient_id: P54321\n• patient_name: Alice Brown\n• patient_dob: None\n• patient_phone_number: 5552971078\n\n"""DATA VALIDATION"""\nIf any CALL METADATA field above (patient_name, patient_dob, or patient_phone_number) is missing, politely ask the patient to provide or confirm the missing information before proceeding.\n\n"""INTENT ONTOLOGY (MUST CHOOSE ONE)"""\n1. schedule_appointment\n2. change_appointment\n3. cancel_appointment\n4. refill_prescription\n5. fill_new_prescription\n6. get_current_prescriptions\n7. lookup_medication_info\n8. lookup_side_effects\n9. check_drug_interactions\n10. evaluate_prior_authorization\n11. transfer_medication_pharmacy\n12. insurance_billing_question\n13. escalate_emergency\n14. general_health_question\n15. unknown / need

In [18]:
lat = cm.get_context("latency_roundtrip", {})

for stage, entries in lat.items():
    # extract all the durations
    durations = [e["dur"] for e in entries]

    # compute summary stats
    count = len(durations)
    average = sum(durations) / count if count else 0
    minimum = min(durations) if count else 0
    maximum = max(durations) if count else 0

    # pretty-print
    print(
        f"⏱️ Stage “{stage}” — "
        f"calls: {count}, "
        f"avg: {average:.3f}s, "
        f"min: {minimum:.3f}s, "
        f"max: {maximum:.3f}s"
    )

⏱️ Stage “tts” — calls: 13, avg: 0.001s, min: 0.001s, max: 0.004s
⏱️ Stage “processing” — calls: 5, avg: 2.720s, min: 0.679s, max: 5.468s


## MED Agent

In [8]:
from rtagents.RTMedAgent.backend.agents.base import RTAgent

In [10]:
med_agent = RTAgent(
    config_path="rtagents/RTMedAgent/backend/agents/agent_store/medication_agent.yaml",
)

2025-06-01 14:46:40,577 - rt_agent - MainProcess - INFO     Loaded agent 'MedicationAgent' | org='GBB AI' | desc='Handles all medication-related requests: refills, new…' | model=gpt-4o-standard | prompt=medication_agent.jinja | tools=['refill_prescription', 'fill_new_prescription', 'get_current_prescriptions', 'lookup_medication_info', 'lookup_side_effects', 'check_drug_interactions'] (base.py:_log_loaded_summary:137)
INFO:rt_agent:Loaded agent 'MedicationAgent' | org='GBB AI' | desc='Handles all medication-related requests: refills, new…' | model=gpt-4o-standard | prompt=medication_agent.jinja | tools=['refill_prescription', 'fill_new_prescription', 'get_current_prescriptions', 'lookup_medication_info', 'lookup_side_effects', 'check_drug_interactions']


Templates found: ['billing_agent.jinja', 'general_healthcare_agent.jinja', 'intent_classifier_agent.jinja', 'medication_agent.jinja', 'non_healthcare_agent.jinja', 'referrals_agent.jinja', 'scheduling_agent.jinja', 'voice_agent_authentication.jinja', 'voice_agent_system.jinja']


In [11]:
med_agent.tools

[{'type': 'function',
  'function': {'name': 'refill_prescription',
   'description': "Refill an existing prescription for a patient's medication.",
   'parameters': {'type': 'object',
    'properties': {'patient_name': {'type': 'string',
      'description': 'Full name of the patient.'},
     'medication_name': {'type': 'string',
      'description': 'Name of the medication to refill.'},
     'pharmacy': {'type': 'string',
      'description': 'Preferred pharmacy name or location (optional).'}},
    'required': ['patient_name', 'medication_name'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'fill_new_prescription',
   'description': 'Add a new prescription for the patient in the system.',
   'parameters': {'type': 'object',
    'properties': {'patient_name': {'type': 'string',
      'description': 'Full name of the patient.'},
     'medication_name': {'type': 'string',
      'description': 'Name of the new medication to add.'},
     'dosage': {'t

## Intent Mapping Agent

In [28]:
from rtagents.RTMedAgent.backend.agents.base import RTAgent

In [29]:
agent_intent_mapping = RTAgent(
    config_path="rtagents/RTMedAgent/backend/agents/agent_store/intent_classifier_agent.yaml",
    session_id="demo123",
)

2025-06-01 00:55:52,607 - rt_agent - MainProcess - INFO     Loaded agent 'IntentClassifierAgent' | org='GBB AI' | desc='Classifies incoming user requests into domain-specific…' | model=gpt-4.1-nano | prompt=intent_classifier_agent.jinja | tools=∅ | session=demo123 (base.py:_log_loaded_summary:143)
INFO:rt_agent:Loaded agent 'IntentClassifierAgent' | org='GBB AI' | desc='Classifies incoming user requests into domain-specific…' | model=gpt-4.1-nano | prompt=intent_classifier_agent.jinja | tools=∅ | session=demo123


Templates found: ['billing_agent.jinja', 'general_healthcare_agent.jinja', 'intent_classifier_agent.jinja', 'medication_agent.jinja', 'non_healthcare_agent.jinja', 'referrals_agent.jinja', 'scheduling_agent.jinja', 'voice_agent_authentication.jinja', 'voice_agent_system.jinja']
Templates found: ['billing_agent.jinja', 'general_healthcare_agent.jinja', 'intent_classifier_agent.jinja', 'medication_agent.jinja', 'non_healthcare_agent.jinja', 'referrals_agent.jinja', 'scheduling_agent.jinja', 'voice_agent_authentication.jinja', 'voice_agent_system.jinja']


In [4]:
agent_intent_mapping.tools

[]

In [30]:
from rtagents.RTMedAgent.backend.services.openai_services import (
    client as az_openai_client,
)
from rtagents.RTMedAgent.backend.agents.prompt_store.prompt_manager import PromptManager

In [31]:
prompt_manager = PromptManager()
prompt = prompt_manager.create_prompt_system_main("intent_classifier_agent.jinja")

Templates found: ['billing_agent.jinja', 'general_healthcare_agent.jinja', 'intent_classifier_agent.jinja', 'medication_agent.jinja', 'non_healthcare_agent.jinja', 'referrals_agent.jinja', 'scheduling_agent.jinja', 'voice_agent_authentication.jinja', 'voice_agent_system.jinja']


In [32]:
messages = (
    [
        {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": "Need a new prescription for my glasses, please.",
        },
    ],
)

In [1]:
from azure.communication.callautomation import (
    CallAutomationClient,
    CallConnectionClient,
    PhoneNumberIdentifier,
    SsmlSource,
    TextSource,
    TranscriptionOptions,
    TranscriptionTransportType,
    RecordingChannel,
    RecordingContent,
    RecordingFormat,
    AzureBlobContainerRecordingStorage,
)

In [3]:
?TranscriptionOptions

Init signature:
TranscriptionOptions(
    *,
    transport_url: str,
    transport_type: Union[str, ForwardRef('TranscriptionTransportType')],
    locale: str,
    start_transcription: bool,
    speech_recognition_model_endpoint_id: Optional[str] = None,
    enable_intermediate_results: Optional[bool] = None,
)
Docstring:     
Configuration of live transcription.

:keyword transport_url: Transport URL for live transcription. Required.
:paramtype transport_url: str
:keyword transport_type: The type of transport to be used for live transcription, eg. Websocket.
 Required. "websocket"
:paramtype transport_type: str or ~azure.communication.callautomation.TranscriptionTransportType
:keyword locale: Defines the locale for the data e.g en-CA, en-AU. Required.
:paramtype locale: str
:keyword speech_recognition_model_endpoint_id: Endpoint where the custom model was deployed.
:paramtype speech_recognition_model_endpoint_id: str
:keyword start_transcription: Determines if the transcription should

In [34]:
import time

test_cases = [
    ("I'd like to reschedule my annual physical.", "Scheduling"),
    ("Could you remind me of my next upcoming appointment?", "Scheduling"),
    ("Please cancel my lab visit on Friday.", "Scheduling"),
    ("Can you help me get a referral for a neurologist?", "Referrals"),
    (
        "My primary care doctor said I need to see a podiatrist. Can you help?",
        "Referrals",
    ),
    ("I need a prescription for allergy medication refilled.", "Medication"),
    ("Does my insurance cover blood work?", "Billing"),
    ("I received a bill, but I think it's a mistake. Who can I talk to?", "Billing"),
    ("How do I update my email address in your system?", "Demographics"),
    ("Can you verify my date of birth on file?", "Demographics"),
    ("What are the symptoms of vitamin D deficiency?", "General Healthcare"),
    ("Can you explain how to manage prediabetes?", "General Healthcare"),
    ("Do I need to worry about mixing Advil and Lisinopril?", "Medication"),
    ("I have a question about safe exercises for arthritis.", "General Healthcare"),
    ("I'd like to move my appointment to next month.", "Scheduling"),
    ("Can you give me directions to the nearest pharmacy?", "Non-Healthcare"),
    ("Play some relaxing music for me.", "Non-Healthcare"),
    (
        "Can you tell me about physical therapy options at your clinic?",
        "General Healthcare",
    ),
    ("Please send me a list of my current medications.", "Medication"),
    (
        "Can I have information about the mental health resources you offer?",
        "General Healthcare",
    ),
]

total_latency = 0
all_outputs = []
num_correct = 0

for utterance, expected in test_cases:
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": utterance},
    ]
    t0 = time.time()
    response = az_openai_client.chat.completions.create(
        messages=messages,
        max_completion_tokens=100,
        temperature=0.0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        model="gpt-4.1-nano",
    )
    latency = time.time() - t0
    total_latency += latency

    output = response.choices[0].message.content.strip()
    # Extract category from output JSON string, robustly (strip, lower, etc.)
    import json

    try:
        output_category = json.loads(output).get("category")
    except Exception:
        output_category = "PARSE_ERROR"

    correct = output_category.lower() == expected.lower()
    if correct:
        num_correct += 1

    all_outputs.append(
        {
            "input": utterance,
            "expected": expected,
            "output": output,
            "parsed_category": output_category,
            "latency_sec": latency,
            "correct": correct,
        }
    )
    print(
        f"User: {utterance}\n  → Model: {output} | Expected: {expected} | Correct: {correct} | Latency: {latency:.2f}s\n"
    )

# Final stats
avg_latency = total_latency / len(test_cases)
print(f"Average Latency: {avg_latency:.2f} seconds")
print(
    f"Accuracy: {num_correct}/{len(test_cases)} ({100.0*num_correct/len(test_cases):.1f}%)"
)

User: I'd like to reschedule my annual physical.
  → Model: {"category": "Scheduling"} | Expected: Scheduling | Correct: True | Latency: 0.45s

User: Could you remind me of my next upcoming appointment?
  → Model: {"category": "Scheduling"} | Expected: Scheduling | Correct: True | Latency: 0.27s

User: Please cancel my lab visit on Friday.
  → Model: {"category": "Scheduling"} | Expected: Scheduling | Correct: True | Latency: 0.27s

User: Can you help me get a referral for a neurologist?
  → Model: {"category": "Referrals"} | Expected: Referrals | Correct: True | Latency: 0.26s

User: My primary care doctor said I need to see a podiatrist. Can you help?
  → Model: {"category": "Referrals"} | Expected: Referrals | Correct: True | Latency: 0.27s

User: I need a prescription for allergy medication refilled.
  → Model: {"category": "Medication"} | Expected: Medication | Correct: True | Latency: 0.25s

User: Does my insurance cover blood work?
  → Model: {"category": "Billing"} | Expected: 